In [1]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

c:\Users\lgrne\OneDrive\Documents\AULAS UFG\TCC\Pedagogical_Conversational_Agent\pca_agent_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#### Chroma Vector Store
#### Para criar do zero, ver chromadb_database_creation.ipynb


db = Chroma(
    persist_directory="./chroma_db",
    collection_name="meus_docs"
)

retriever = db.as_retriever()

C:\Users\lgrne\AppData\Local\Temp\ipykernel_17140\1083490982.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(


In [3]:
llm = OllamaLLM(model="LLama3.1:8b")

prompt = ChatPromptTemplate.from_template(
"""
Você é um ACP - um Agente Conversacional Pedagógico especializado em ajudar estudantes com dúvidas relacionadas a conteúdos acadêmicos.
Sua função é fornecer respostas claras, concisas e informativas com base no material de estudo fornecido pelo contexto.
Suas respostas devem ser sempre em português brasileiro e devem usar um tom leve.
            
Contexto:
{context}

Pergunta:
{question}
""")

chain = (
        prompt
        | llm
        | StrOutputParser()
    )


In [4]:
#### Testando prompt com contexto
#### Na teoria, essa resposta será mais embasada nos documentos da RAG
#### PS: Sem GPU leva mais de 1 minuto para responder

def rag_pipeline(question: str):
    docs = retriever.invoke(question)
    context = "\n\n".join([d.page_content for d in docs])

    chain = (
        prompt
        | llm
        | StrOutputParser()
    )

    return chain.invoke({"context": context, "question": question})

response = rag_pipeline("Quais as diferenças entre um ML Engineer e um Data Scientist?")
print(response)

Oi, não está na lista de perguntas fornecidas no contexto, mas eu vou fazer o meu melhor para responder de forma clara e concisa!

Um ML Engineer (Engenheiro de Aprendizado de Máquina) e um Data Scientist (Cientista de Dados) são profissionais que trabalham em áreas relacionadas, mas com focos diferentes.

**Data Scientist:**
Um Data Scientist é responsável por extrair conhecimento a partir dos dados coletados. Eles utilizam técnicas de análise estatística e processamento de linguagem natural para entender os padrões nos dados e criar modelos que predizem resultados ou identifiquem tendências.

**ML Engineer:**
Um ML Engineer, por outro lado, é responsável pela implementação dos modelos de aprendizado de máquina em produção. Eles se concentram em desenvolver sistemas capazes de aprender e melhorar ao longo do tempo, utilizando técnicas de machine learning.

Em resumo, um Data Scientist foca em entender os dados e criar modelos que predizem resultados, enquanto um ML Engineer se concent

In [5]:
#### Testando resposta sem contexto
#### Essa resposta deverá ser mais genérica

def rag_pipeline(question: str):
    docs = retriever.invoke(question)
    # context = "\n\n".join([d.page_content for d in docs])
    context = ''

    chain = (
        prompt
        | llm
        | StrOutputParser()
    )

    return chain.invoke({"context": context, "question": question})

response = rag_pipeline("Quais as diferenças entre um ML Engineer e um Data Scientist?")
print(response)

Oi, querido estudante! Vamos lá, vamos esclarecer essa dúvida!

Um ML Engineer (Engenheiro de Máquina de Aprendizado) e um Data Scientist são profissionais que trabalham com dados, mas têm papéis e focos diferentes.

**ML Engineer**:
- Foco em desenvolver e implementar modelos de aprendizado de máquina para solucionar problemas específicos.
- Responsável por construir e depurar sistemas de ML, garantindo a eficiência e escalabilidade da aplicação.
- Trabalha com tecnologias como Python, TensorFlow, PyTorch e frameworks similares.

**Data Scientist**:
- Foco em entender os dados, extrair insights valiosos e comunicar esses achados para tomar decisões informadas.
- Responsável por explorar, modelar e analisar grandes conjuntos de dados para descobrir padrões e tendências.
- Trabalha com linguagens como Python, R ou SQL, além de ferramentas de visualização de dados.

Em resumo: o ML Engineer é mais focado no desenvolvimento e implementação dos modelos de aprendizado, enquanto o Data Scien